**PART 07 최신 기출문제**
>**기출문제 제9회**
>>**01 작업형 제1유형**

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) # 모든 칼럼이 출력되게 조절
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/9_1_1.csv')
print(df.head( ))

In [ ]:
df['총대출액'] = df['금액1'] + df['금액2']

In [ ]:
dfgender = df.groupby(['year', 'gender', '지역코드'])['총대출액'].sum( ).reset_index( )

In [ ]:
df_pivot = dfgender.pivot_table(index = ['year', '지역코드'], columns = 'gender', values = '총대출액', fill_value = 0)

In [ ]:
df_pivot

In [ ]:
df_pivot['abs_diff'] = abs(df_pivot[0] - df_pivot[1])

print('지역코드 :', df_pivot.index[0][1])

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/9_1_2.csv')
df.head()

In [ ]:
df1 = df[df["구분"] == "검거건수"].set_index("연도").drop(columns="구분")
df2 = df[df["구분"] == "발생건수"].set_index("연도").drop(columns="구분")

In [ ]:
dfratio = df1 / df2

In [ ]:
dfmax = dfratio.idxmax(axis = 1) # 각 연도별 최대 검거율을 가진 범죄유형
dfmax_reset = dfmax.reset_index( ).rename({0: '범죄유형'}, axis = 1)

In [ ]:
df1_melt = df1.reset_index( ).melt(id_vars = "연도", var_name = "범죄유형", value_name = "검거건수")
dffinal = pd.merge(dfmax_reset, df1_melt, on = ['연도', '범죄유형'], how = 'left')
total_sum = dffinal['검거건수'].sum( )
print("총 검거건수 합계 = ", total_sum)

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/9_1_3.csv')
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df["평균만족도"] = df["평균만족도"].fillna(df["평균만족도"].mean())

In [ ]:
mean_tenure = (
  df.groupby(["부서", "등급"])["근속연수"]
  .mean()
  .apply(np.floor)
  .reset_index()
  .rename(columns = {"근속연수": "평균근속연수"})
)

# merge로 결측치 대치
df = df.merge(mean_tenure, on = ["부서", "등급"], how = "left")
df["근속연수"] = np.where(
  df["근속연수"].isnull( ), # 조건: 근속연수가 결측치인 경우
  df["평균근속연수"], # 결측치일 때 대치값
  df["근속연수"] # 결측치가 아닐 때 기존 값 유지
)
df.drop(columns = ["평균근속연수"], inplace = True)

In [ ]:
df.isna().sum()

In [ ]:
A = df.loc[(df["부서"] == "HR") & (df["등급"] == "A"), "근속연수"].mean()

In [ ]:
B = df.loc[(df["부서"] == "Sales") & (df["등급"] == "B"), "교육참가횟수"].mean()

In [ ]:
result = A + B
print("A (평균 근속연수):", A)
print("B (평균 교육참가횟수):", B)
print("A + B:", result)

>>**02 작업형 제2유형**

In [1]:
import pandas as pd
import numpy as np
# 데이터 로드
train = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/9_2_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/9_2_test.csv')

In [ ]:
print(train.info( ))

In [ ]:
print(test.info())

In [2]:

train_X = train.drop(['라벨'], axis = 1)
train_y = train['라벨']
test_X = test.drop(['라벨'], axis = 1)
test_y = test['라벨']

In [3]:
from sklearn.model_selection import train_test_split
# 데이터 분할
train_X, valid_X, train_y, valid_y = train_test_split(train_X, train_y, test_size = 0.3, random_state = 1)

In [4]:
cat_columns = train_X.select_dtypes('object').columns.to_list( )
# num_columns = train_X.select_dtypes('number').columns.to_list( )

In [5]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder(sparse_output = False, handle_unknown = 'ignore')

In [6]:
train_X_preprocessed = onehotencoder.fit_transform(train_X[cat_columns])
valid_X_preprocessed = onehotencoder.transform(valid_X[cat_columns])
test_X_preprocessed = onehotencoder.transform(test_X[cat_columns])

In [7]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state = 1)
rf.fit(train_X_preprocessed, train_y)

RandomForestClassifier(random_state=1)

In [8]:
from sklearn.metrics import f1_score
pred_val = rf.predict(valid_X_preprocessed)
print('valid f1-macro:' , f1_score(valid_y, pred_val, average = 'macro'))

valid f1-macro: 0.3099787685774947


In [ ]:
test_pred = rf.predict(test_X_preprocessed)
test_pred = pd.DataFrame(test_pred, columns = ['pred'])

In [ ]:
result = pd.concat([test_id, test_pred], axis = 1)
print(result.head(2))

In [ ]:
result.to_csv('9_2_result.csv', index = False) # 시험에서 제시되는 파일명 작성할 것

In [ ]:
train_X_full = np.concatenate([train_X_preprocessed, valid_X_preprocessed], axis = 0)
train_y_full = np.concatenate([train_y, valid_y], axis = 0)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'max_depth': [10, 20, 30],
 'min_samples_split': [2, 5, 10]}

rf = RandomForestClassifier(random_state = 1)
rf_search = GridSearchCV(estimator = rf,
  param_grid = param_grid,
  cv = 3,
  scoring = 'f1_macro')

rf_search.fit(train_X_full, train_y_full)
GridSearchCV(cv = 3, estimator = RandomForestClassifier(random_state = 1),
  param_grid = {'max_depth': [10, 20, 30],
  'min_samples_split': [2, 5, 10]},
  scoring = 'f1_macro')

print('교차검증 MAE-score:', rf_search.best_score_)

In [ ]:
test_pred2 = rf_search.predict(test_X_preprocessed)
test_pred2 = pd.DataFrame(test_pred2, columns = ['pred'])

In [ ]:
result = pd.concat([test_id, test_pred2], axis = 1)

In [ ]:
train = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/9_2_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/9_2_test.csv')
test_id = test['ID']

train_X = train.drop(['라벨'], axis = 1)
train_y = train['라벨']
test_X = test.drop(['라벨'], axis = 1)
test_y = test['라벨']

In [ ]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder(sparse_output = False, handle_unknown = 'ignore')

cat_columns = train_X.select_dtypes('object').columns.to_list( )
# num_columns = train_X.select_dtypes('number').columns.to_list( )

# ColumnTransformer를 활용한 데이터 전처리
from sklearn.compose import ColumnTransformer
c_transformer = ColumnTransformer(
  transformers = [
  ('cat', onehotencoder, cat_columns),
  # ('num', imputer, num_columns)
  ], remainder = 'passthrough'
)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
pipe_rf = Pipeline(
 [
  ("preprocess", c_transformer),
  ("regressor", RandomForestClassifier(random_state = 1))
 ]
)

In [ ]:
param_grid = {'regressor__max_depth': [10, 20, 30],
  'regressor__min_samples_split': [2, 5, 10]}
rf_search_pipe = GridSearchCV(estimator = pipe_rf,
    param_grid = param_grid,
    cv = 3,
    scoring = 'f1_macro')
rf_search_pipe.fit(train_X, train_y)

In [ ]:
print('교차검증 f1-macro:', rf_search_pipe.best_score_)

In [ ]:
test_pred3 = rf_search_pipe.predict(test_X)
test_pred3 = pd.DataFrame(test_pred3, columns = ['pred'])

In [ ]:
result = pd.concat([test_id, test_pred3], axis = 1)
print(result.head(2))

In [ ]:
result.to_csv('9_2_result.csv', index = False) # 시험에서 제시되는 파일명 작성할 것

>>**03 작업형 제3유형**

In [ ]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/9_3_1.csv')
train = df[df['id'] <= 140]
test = df[df['id'] > 140]

In [ ]:
from statsmodels.formula.api import ols
formula = "design ~ tenure + f2 + f3 + f4 + f5"
model = ols(formula, data = train).fit( )

In [ ]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 design   R-squared:                       0.837
Model:                            OLS   Adj. R-squared:                  0.831
Method:                 Least Squares   F-statistic:                     137.4
Date:                Sat, 07 Jun 2025   Prob (F-statistic):           5.91e-51
Time:                        02:26:09   Log-Likelihood:                -421.42
No. Observations:                 140   AIC:                             854.8
Df Residuals:                     134   BIC:                             872.5
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.5393      2.584      0.983      0.327      -2.571       7.649
tenure         0.5411      0.077      6.993      0.000       0.388       0.694
f2             0.2081      0.043      4.800      0.000       0.122       0.294
f3            -0.4736      0.853     -0.556      0.579      -2.160       1.213
f4             0.3931      0.016     24.950      0.000       0.362       0.424
f5             0.0144      0.073      0.197      0.844      -0.130       0.159
==============================================================================
Omnibus:                        6.999   Durbin-Watson:                   1.992
Prob(Omnibus):                  0.030   Jarque-Bera (JB):                6.709
Skew:                          -0.523   Prob(JB):                       0.0349
Kurtosis:                       3.234   Cond. No.                         457.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
p_values = model.pvalues

In [ ]:
non_significant_vars = (p_values[1:] >= 0.05).sum( )
print("유의하지 않은 설명변수의 개수:", non_significant_vars)

유의하지 않은 설명변수의 개수: 2


In [ ]:
y_pred = model.predict(train)
y_real = train['design']

from scipy.stats import pearsonr
correlation, a= pearsonr(y_pred, y_real)
correlation, a
#print("상관계수:", correlation)

(np.float64(0.914750154303963), np.float64(3.571926429030428e-56))

In [ ]:
y_pred_series = pd.Series(y_pred)
y_real_series = pd.Series(y_real)

correlation = y_pred_series.corr(y_real_series)
print("상관계수:", correlation)

상관계수: 0.9147501543039637


In [ ]:
y_pred_test = model.predict(test)
from sklearn.metrics import root_mean_squared_error
rmse = root_mean_squared_error(test['design'], y_pred_test)
print("RMSE:", rmse)

RMSE: 4.396152958589427


In [ ]:
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/9_3_2.csv')

In [ ]:
from statsmodels.formula.api import logit
formula = "churn ~ col1 + col2 + Phone_Service + Tech_Insurance"
model = logit(formula, data=df).fit()

Optimization terminated successfully.
         Current function value: 0.640721
         Iterations 5


In [ ]:
col1_pvalue = round(model.pvalues['col1'], 3)
print("col1의 p-value:", col1_pvalue)

col1의 p-value: 0.0


In [ ]:
odds_ratio_phone_service = round(np.exp(model.params['Phone_Service']), 3)
print("Phone_Service의 오즈비:", odds_ratio_phone_service)

Phone_Service의 오즈비: 1.867


In [ ]:
predicted_probs = model.predict(df) # 예측 확률 계산
num_customers_above_03 = sum(predicted_probs > 0.3) # 확률 0.3 초과 고객 수
print("확률 0.3 이상 고객 수:", num_customers_above_03)

확률 0.3 이상 고객 수: 450
